In [34]:
# DAY01
f = open('input/01.txt', 'r')
lines = [line.strip() for line in f.readlines()]

MAPPING = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9,
}

def get_digit(s, dir=1, count_words=False):
    p = 0 if dir == 1 else len(s) - 1
    while p >= 0 and p < len(s):
        if s[p].isdigit():
            return s[p]
        if count_words:
            for k, v in MAPPING.items():
                if s[p:].startswith(k):
                    return str(v)
        p += dir

def get_number(line, count_words=False):
    n = int(get_digit(line, 1, count_words)) * 10 + int(get_digit(line, -1, count_words))
    return n

res1 = sum(get_number(l) for l in lines)
print(res1)

res2 = sum(get_number(l, True) for l in lines)
print(res2)

55130
54985


In [35]:
# DAY02
f = open('input/02.txt', 'r')
lines = [line.strip() for line in f.readlines()]


def parse_turn(line):
    parts = line.split(',')
    res = [0, 0, 0]
    for p in parts:
        p = p.strip()
        n, color = p.split(' ')
        color = color.strip()
        if color == 'red':
            res[0] += int(n)
        if color == 'green':
            res[1] += int(n)
        if color == 'blue':
            res[2] += int(n)
    return res


def parse_line(line):
    line = line.split(': ')[1]
    return [parse_turn(t) for t in line.split(';')]


def is_game_possible(game, target):
    for r, g, b in game:
        if r > target[0] or g > target[1] or b > target[2]:
            return False
    return True


def get_min_target(game):
    min_r = max([r for r, _, _ in game])
    min_g = max([g for _, g, _ in game])
    min_b = max([b for _, _, b in game])
    return [min_r, min_g, min_b]


games = [parse_line(l) for l in lines]

TARGET = [12, 13, 14]

res1 = 0
for i in range(len(games)):
    if is_game_possible(games[i], TARGET):
        res1 += i + 1

print(res1)

min_targets = [get_min_target(g) for g in games]
res2 = sum(max(1, r) * max(1, g) * max(1, b) for r, g, b in min_targets)
print(res2)


2913
55593


In [29]:
# DAY03
f = open('input/03.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def is_symbol(c):
    return c is not None and not c.isdigit() and c != '.'


OFFSETS = [
    [-1, -1], [0, -1], [1, -1],
    [-1, 0], [1, 0],
    [-1, 1], [0, 1], [1, 1]]


def get_at(lines, x, y):
    h = len(lines)
    w = len(lines[0])
    if x >= 0 and x < w and y >= 0 and y < h:
        return lines[y][x]
    return None

def has_symbol_neighbor(lines, i, j):
    h = len(lines)
    w = len(lines[0])
    for dx, dy in OFFSETS:
        x = i + dx
        y = j + dy
        if is_symbol(get_at(lines, x, y)):
            return True
    return False

def get_gear_neighbors(lines, i, j):
    res = set()
    h = len(lines)
    w = len(lines[0])
    for dx, dy in OFFSETS:
        x = i + dx
        y = j + dy
        c = get_at(lines, x, y)
        if c == '*':
            res.add((x, y))
    return res

def extract_nums(lines):
    h = len(lines)
    w = len(lines[0])
    
    is_part = False
    n = 0
    num_digits = 0
    nums = []
    gears = set()
    for j in range(h):
        for i in range(w):
            c = lines[j][i]
            
            if c.isdigit():
                n = n * 10 + int(c)
                num_digits += 1
                is_part = is_part or has_symbol_neighbor(lines, i, j)
                g = get_gear_neighbors(lines, i, j)
                if g is not None:
                    gears = gears.union(g)
                
            if (not c.isdigit() or i == w - 1) and num_digits > 0:
                nums.append((n, is_part, gears))
                is_part = False
                n = 0
                num_digits = 0
                gears = set()
    return nums


def extract_gears(nums):
    gears = {}
    for _, _, gs in nums:
        for g in gs:
            if g in gears:
                gears[g] += 1
            else:
                gears[g] = 1
                
    gears = {g: 1 for g, count in gears.items() if count == 2}
    
    for n, is_part, gs in nums:
        for g in gs:
            if g in gears:
                gears[g] *= n
    return gears
    

nums = extract_nums(lines)
res1 = sum(n for n, is_part, _ in nums if is_part)
print(res1)

gears = extract_gears(nums)
res2 = sum(gears.values())
print(res2)

530849
84900879


In [30]:
# DAY04
f = open('input/04.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_card(line):
    line = line.split(": ")[1]
    n1, n2 = line.split("|")
    player_nums = [int(x.strip()) for x in n2.strip().split()]
    winning_nums = [int(x.strip()) for x in n1.strip().split()]
    return (winning_nums, player_nums)

def get_num_matches(card):
    common = set(card[0]).intersection(set(card[1]))
    return len(common)

def get_score(card):
    num_matches = get_num_matches(card)
    return int(2 ** num_matches / 2)

def eval_num_copies(cards):
    ncards = len(cards)
    copies = [1] * ncards
    for i in range(ncards):
        nm = get_num_matches(cards[i])
        for j in range(i + 1, min(i + 1 + nm, ncards)):
            copies[j] += copies[i]
    return copies
        
cards = [parse_card(line) for line in lines]

res1 = sum(get_score(c) for c in cards)
print(res1)

copies = eval_num_copies(cards)
res2 = sum(copies)
print(res2)
        
    

25651
19499881


In [31]:
# DAY05
text = open('input/05.txt', 'r').read()

def parse_ints(line):
    return [int(x) for x in line.split()]
    
def parse_mapping(text):
    parts = text.split('\n')
    return [parse_ints(p) for p in parts[1:] if p != '']
    
def parse_problem(text):
    parts = text.split('\n\n')
    seeds = parse_ints(parts[0].split(': ')[1])
    mappings = [parse_mapping(p) for p in parts[1:]]
    return seeds, mappings

def map_seed(seed, mappings):
    for m in mappings:
        for d, s, l in m:
            if seed >= s and seed < s + l:
                seed = d + seed - s
                break
    return seed

def map_range(range, mapping):
    ranges = set()
    ranges.add(range)
    res = []
    while len(ranges) > 0:
        r = ranges.pop()
        ranges.add(r)
        mapped = False
        for d, s, l in mapping:
            rs, rl = r
            if (s < rs + rl) and (s + l > rs):
                p1 = d + max(s, rs) - s
                p2 = d + min(s + l, rs + rl) - s
                res.append((p1, p2 - p1))
                mapped = True
                # clip the range:
                ranges.remove(r)
                if rs < s:
                    ranges.add((rs, s - rs))
                if rs + rl > s + l:
                    ranges.add((s + l, rs + rl - s - l))
                break
        if not mapped:
            break
    res += list(ranges)
    return res

def map_ranges(ranges, mappings):
    for mapping in mappings:
        res = []
        for r in ranges:
            res += map_range(r, mapping)
            ranges = res
    return ranges

seeds, mappings = parse_problem(text)

res1 = min(map_seed(s, mappings) for s in seeds)
print(res1)

ranges = [(seeds[i * 2], seeds[i * 2 + 1]) for i in range(int(len(seeds)/2))]
res2 = min(x for x, _ in map_ranges(ranges, mappings))
print(res2)

331445006
6472060


In [32]:
# DAY06
import math
from operator import mul
from functools import reduce

f = open('input/06.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_ints(line):
    return [int(x.strip()) for x in line.split()]

def parse_input(lines):
    t = parse_ints(lines[0].split(':')[1])
    d = parse_ints(lines[1].split(':')[1])
    return [(t[i], d[i]) for i in range(len(t))]

def get_roots_dist(t, d):
    det = t * t - 4 * d
    if det < 0:
        return 0
    ds = math.sqrt(det)
    r1 = math.ceil(0.5 * (ds + t))
    r2 = math.floor(0.5 * (t - ds))
    return abs(r1 - r2 - 1)
    
races = parse_input(lines)
dists = [get_roots_dist(t, d) for t, d in races]
res1 = reduce(mul, dists, 1)
print(res1)

lines2 = parse_input([l.replace(' ', '') for l in lines])
res2 = get_roots_dist(races2[0][0], races2[0][1])
print(res2)

1083852
23501589


In [57]:
# DAY07
from functools import cmp_to_key

f = open('input/07.txt', 'r')
lines = [line.strip() for line in f.readlines()]


def parse_card(line):
    p = line.split()
    return (p[0], int(p[1]))


def get_hist(card):
    res = {}
    for c in card:
        if c in res:
            res[c] += 1
        else:
            res[c] = 1
    return res


def get_hand_id_raw(card):
    h = get_hist(card)
    f = sorted(h.values(), reverse=True)
    if f[0] == 5:
        return 0
    elif f[0] == 4:
        return 1
    elif f[0] == 3 and f[1] == 2:
        return 2
    elif f[0] == 3:
        return 3
    elif f[0] == 2 and f[1] == 2:
        return 4
    elif f[0] == 2:
        return 5
    elif len(f) == 5:
        return 6
    else:
        return 7


def get_hand_id(card, use_joker=False):
    if not use_joker:
        return get_hand_id_raw(card)
    return min(get_hand_id_raw(card.replace('J', c)) for c in get_hist(card).keys())


def compare_cards(ca,  cb, ranks_map, use_joker=False):
    a, _ = ca
    b, _ = cb
    h1 = get_hand_id(a, use_joker)
    h2 = get_hand_id(b, use_joker)
    if h1 < h2:
        return -1
    elif h1 > h2:
        return 1
    for i in range(len(a)):
        r1 = ranks_map[a[i]]
        r2 = ranks_map[b[i]]
        if r1 < r2:
            return -1
        elif r1 > r2:
            return 1
    return 0


RANKS = "AKQJT98765432"
RANKS_MAP = {RANKS[i]: i for i in range(len(RANKS))}


def compare_cards1(ca, cb):
    return compare_cards(ca, cb, RANKS_MAP, False)


RANKS2 = "AKQT98765432J"
RANKS_MAP2 = {RANKS2[i]: i for i in range(len(RANKS2))}


def compare_cards2(ca, cb):
    return compare_cards(ca, cb, RANKS_MAP2, True)


def get_score(cards):
    return sum(cards[i][1] * (i + 1) for i in range(len(cards)))


cards = [parse_card(line) for line in lines]

cards = sorted(cards, key=cmp_to_key(compare_cards1), reverse=True)
res1 = get_score(cards)
print(res1)

cards = sorted(cards, key=cmp_to_key(compare_cards2), reverse=True)
res2 = get_score(cards)
print(res2)

252052080
252898370


In [1]:
# DAY08
import math
import functools 

f = open('input/08.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_node(line):
    parts = line.split(' = ')
    return parts[0], parts[1][1:-1].split(', ')

def lcm(a, b):
    return abs(a*b) // math.gcd(a, b)

def get_nsteps(n='AAA', lastz=True):
    i = 0
    while True:
        s = dirs[i % len(dirs)]
        idx = 0 if s == 'L' else 1
        n = nodes[n][idx]
        i += 1
        if (not lastz and n == 'ZZZ') or (lastz and n[-1] == 'Z'):
            return i

dirs = lines[0]
nodes = [parse_node(line) for line in lines[2:]]
nodes = {n: lr for n, lr in nodes}

res1 = get_nsteps('AAA', False)
print(res1)

start_nodes = [x for x in nodes.keys() if x[-1] == 'A']
nsteps = [get_nsteps(n, True) for n in start_nodes]
res2 = functools.reduce(lambda a, b: lcm(a, b), nsteps)
print(res2)

12737
9064949303801


In [23]:
# DAY09

f = open('input/09.txt', 'r')


def parse_ints(line):
    return [int(x.strip()) for x in line.split()]


def get_derivatives(nums):
    n = len(nums)
    res = [0] * (n - 1)
    for i in range(n - 1):
        res[i] = nums[i + 1] - nums[i]
    return res
    

def extrapolate(nums):
    derivatives = [nums]
    n = nums
    while True:
        d = get_derivatives(n)
        if all(x == 0 for x in d):
            res1, res2 = 0, 0
            nd = len(derivatives)
            for i in range(nd):
                res1 = derivatives[nd - 1 - i][0] - res1
                res2 += derivatives[nd - 1 - i][-1]
            return res1, res2
        derivatives.append(d)
        n = d

nums = [parse_ints(line.strip()) for line in f.readlines()]

extr = [extrapolate(n) for n in nums]
res1 = sum(b for _, b in extr)
print(res1)

res2 = sum(a for a, _ in extr)
print(res2)

1782868781
1057


In [141]:
# DAY10

f = open('input/10_test2.txt', 'r')
field = [line.strip() for line in f.readlines()]

DIRS = [(0, 1), (1, 0), (-1, 0), (0, -1)]
SYMS = {
    '|': [(0, -1), (0, 1)],
    '-': [(-1, 0), (1, 0)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, -1), (-1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, 1), (1, 0)],
}

def find_start(field):
    w = len(field[0])
    h = len(field)
    for j in range(h):
        for i in range(w):
            if field[j][i] == 'S':
                return i, j


def find_next(field, start):
    w, h = len(field[0]), len(field)
    x, y = start
    for dx, dy in DIRS:
        px, py = x + dx, y + dy
        if px < 0 or px >= w or py < 0 or py >= h:
            continue
        c = field[py][px]
        if not c in SYMS:
            continue
        for kx, ky in SYMS[c]:
            nx, ny = px + kx, py + ky
            if (nx, ny) == (x, y):
                yield px, py

def get_sym(field, pos):
    x, y = pos
    c = field[y][x]
    if c != 'S':
        return c
    adj = sorted([(px - x, py - y) for px, py in find_next(field, pos)])
    for k, v in SYMS.items():
        if adj == sorted(v):
            return k

def traverse(field):
    w, h = len(field[0]), len(field)
    prev = find_start(field)
    x, y = list(find_next(field, prev))[0]
    yield prev
    while True:
        c = field[y][x]
        for dx, dy in SYMS[c]:
            px, py = x + dx, y + dy
            if (px, py) != prev:
                prev = x, y
                x, y = px, py
                break
        yield prev
        if field[y][x] == 'S' or (x, y) == prev:
            return

def get_inside(field, path):
    w, h = len(field[0]), len(field)
    res = 0
    spath = set(path)
    for j in range(h):
        inside = False
        i = 0
        while i < w:
            c = get_sym(field, (i, j))
            if (i, j) in spath:
                if c == '|':
                    inside = not inside
                elif c == 'F':
                    inside = not inside
                    while get_sym(field, (i, j)) not in ['J', '7']:
                        i += 1
                    if get_sym(field, (i, j)) == '7':
                        inside = not inside
                elif c == 'L':
                    inside = not inside
                    while get_sym(field, (i, j)) not in ['J', '7']:
                        i += 1
                    if get_sym(field, (i, j)) == 'J':
                        inside = not inside
            else:
                if inside:
                    yield i, j
            i += 1

path = [p for p in traverse(field)]
res1 = int(len(path) / 2)
print(res1)
    
inside = list(get_inside(field, path))
res2 = len(inside)
print(res2)

def draw_field(field, path, inside):
    SUBST = {
        '|': '│',
        '-': '─',
        'L': '└',
        'J': '┘',
        '7': '┐',
        'F': '┌',
    }
    spath = set(path)
    sinside = set(inside)
    for j in range(len(field)):
        line = field[j]
        chars = [c for c in line]
        for i in range(len(chars)):
            c = chars[i]
            if c == 'S':
                continue
            if (i, j) not in spath:
                chars[i] = '█' if (i, j) in sinside else ' '
            else:
                chars[i] = SUBST[c]
        print(''.join(chars))

draw_field(field, path, inside)

70
8
 ┌────┐┌┐┌┐┌┐┌─┐    
 │┌──┐││││││││┌┘    
 ││ ┌┘││││││││└┐    
┌┘└┐└┐└┘└┘││└┘█└─┐  
└──┘ └┐███└┘S┐┌─┐└┐ 
    ┌─┘██┌┐┌┘│└┐└┐└┐
    └┐█┌┐││└┐│█└┐└┐│
     │┌┘└┘│┌┘│┌┐│ └┘
    ┌┘└─┐ ││ ││││   
    └───┘ └┘ └┘└┘   


In [161]:
# DAY11

f = open('input/11.txt', 'r')
lines = [line.strip() for line in f.readlines()]


def parse_field(lines):
    h = len(lines)
    w = len(lines[0])
    res = set()
    for j in range(h):
        for i in range(w):
            if lines[j][i] == '#':
                res.add((i, j))
    return res


def manhattan_dist(a, b):
    ax, ay = a
    bx, by = b
    return abs(ax - bx) + abs(ay - by)


def expand_field(field, factor=2):
    sf = sorted(field)
    res = set()
    lastx = sf[0][0]
    delta = 0
    for x, y in sf:
        if x - lastx > 1:
            delta += (factor - 1) * (x - lastx - 1)
        res.add((x + delta, y))
        lastx = x
    return res

def swap_coords(field):
    res = set()
    for x, y in field:
        res.add((y, x))
    return res

def get_dists(field):
    coords = list(field)
    nc = len(coords)
    res = 0
    for i in range(nc):
        for j in range(i + 1, nc):
            res += manhattan_dist(coords[i], coords[j])
    return res

def solve(field, factor=2):
    field = expand_field(field, factor)
    field = swap_coords(expand_field(swap_coords(field), factor))
    return get_dists(field)

field = parse_field(lines)
res1 = solve(field)
print(res1)

res2 = solve(field, 1000000)
print(res2)

9609130
702152204842


In [ ]:
%%time

f = open('input/12.txt', 'r')
lines = [line.strip() for line in f.readlines()]

def parse_line(line):
    parts = line.split()
    return list(parts[0]), [int(x.strip()) for x in parts[1].strip().split(",")]

mem = {}

def get_num_subst(syms, ranges, syms_offs=0, range_offs=0, cont=False):
    global mem
    nr = len(ranges)
    if syms_offs == len(syms) and (range_offs == nr or 
                                   (range_offs == nr - 1 and ranges[range_offs] == 0)):
        return 1
    if syms_offs == len(syms):
        return 0
    c = syms[syms_offs]

    key = "".join(syms[syms_offs:]) + "|" + ",".join(
        str(x) for x in ranges[range_offs:]) + ("~" if cont else "")
    if key in mem:
        return mem[key]
        
    if c == '#':
        if range_offs == nr or ranges[range_offs] == 0:
            return 0
        ranges[range_offs] -= 1
        n = get_num_subst(syms, ranges, syms_offs + 1, range_offs, True)
        ranges[range_offs] += 1
        mem[key] = n
        return n
    elif c == '.':
        if range_offs < nr and ranges[range_offs] != 0 and cont:
            return 0
        if range_offs < nr and ranges[range_offs] == 0:
            n = get_num_subst(syms, ranges, syms_offs + 1, range_offs + 1, False)
        else:
            n = get_num_subst(syms, ranges, syms_offs + 1, range_offs, False)
        mem[key] = n
        return n
    elif c == '?':
        syms[syms_offs] = '#'
        n1 = get_num_subst(syms, ranges, syms_offs, range_offs, cont)
        syms[syms_offs] = '.'
        n2 = get_num_subst(syms, ranges, syms_offs, range_offs, cont)
        syms[syms_offs] = '?'
        mem[key] = n1 + n2
        return n1 + n2

def get_num_subst1(sym_ranges, ranges, sym_ranges_offs=0, ranges_offs=0):
    if sym_ranges_offs == len(sym_ranges):
        return int(ranges_offs == len(ranges))
    syms = sym_ranges[sym_ranges_offs]
    res = 0
    n = 0
    while ranges_offs + n <= len(ranges):
        ns = get_num_subst(syms, ranges[ranges_offs:ranges_offs + n])
        nsrest = 0
        if ns > 0:
            nsrest = get_num_subst1(sym_ranges, ranges, sym_ranges_offs + 1, ranges_offs + n)
        res += ns * nsrest
        n += 1
    return res


def expand(e, n=5):
    return list("?".join(["".join(e[0])] * n)), e[1] * n


def split_sym_ranges(e):
    rs = [list(p) for p in "".join(e).split('.')]
    return [r for r in rs if len(r) > 0]
    

entries = [parse_line(line) for line in lines]

nsubst = [get_num_subst(syms, ranges) for syms, ranges in entries]
res1 = sum(nsubst)
print(f"Answer 1: {res1}")

exp = [expand(e, 5) for e in entries]
nsubst2 = []

for i in range(len(exp)):
    syms, ranges = exp[i]
    syms_str = "".join(syms)
    print(f"{i:0>3}/{len(exp)}: {syms_str} | {ranges} ")
    n = get_num_subst1(split_sym_ranges(syms), ranges)
    nsubst2.append(n)
    
res2 = sum(nsubst2)
print(f"Answer 2: {res2}")

Answer 1: 6852
000/1000: #.?#.#???????...??##?#.?#.#???????...??##?#.?#.#???????...??##?#.?#.#???????...??##?#.?#.#???????...??## | [1, 2, 2, 1, 1, 4, 1, 2, 2, 1, 1, 4, 1, 2, 2, 1, 1, 4, 1, 2, 2, 1, 1, 4, 1, 2, 2, 1, 1, 4] 
001/1000: ???#???.?#??????????#???.?#??????????#???.?#??????????#???.?#??????????#???.?#?????? | [1, 2, 2, 3, 2, 1, 2, 2, 3, 2, 1, 2, 2, 3, 2, 1, 2, 2, 3, 2, 1, 2, 2, 3, 2] 
002/1000: ???????#??.???#?????????#??.???#?????????#??.???#?????????#??.???#?????????#??.???#? | [2, 2, 4, 3, 2, 2, 4, 3, 2, 2, 4, 3, 2, 2, 4, 3, 2, 2, 4, 3] 
003/1000: .???????#???.???????#???.???????#???.???????#???.???????#?? | [4, 1, 4, 1, 4, 1, 4, 1, 4, 1] 
004/1000: ?????#????.??????#????.??????#????.??????#????.??????#????. | [1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1] 
005/1000: #.?#???#???.?#.?#.?#???#???.?#.?#.?#???#???.?#.?#.?#???#???.?#.?#.?#???#???.?#. | [1, 3, 1, 1, 2, 1, 3, 1, 1, 2, 1, 3, 1, 1, 2, 1, 3, 1, 1, 2, 1, 3, 1, 1, 2] 
006/1000: ???#?.#.?#??????#????#?.#.?#??????#????#?